In [ ]:
import torchvision.models as models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.optim as optim

In [ ]:
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, indices, photo_path, transform, train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.train = train
        self.indices = indices
        self.photo_path = photo_path

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        filename = self.dataframe.loc[original_idx, 'id']
        file_path = os.path.join(self.photo_path, str(filename))

        image = Image.open(file_path).convert("RGB")
        image = self.transform(image)

        if self.train:
            label = torch.tensor(self.dataframe['mapped'].loc[original_idx], dtype=torch.long)
            return image, label
        else:
            return image


In [ ]:
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])

In [ ]:
class Resnet(nn.Module):
    def __init__(self, num_classes=8):
        super(Resnet, self).__init__()

        self.model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)

        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

class EfficientNet(nn.Module):
  def __init__(self, num_classes=8):
    super(EfficientNet, self).__init__()

    self.model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.IMAGENET1K_V1)

    self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
  def forward(self, x):
    return self.model(x)
class EfficientNetLarge(nn.Module):
    def __init__(self, num_classes=8):
        super(EfficientNetLarge, self).__init__()

        self.model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)

        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [ ]:
from sklearn.model_selection import train_test_split
# X_train -- ТВОЙ ДАТАСЕТ
train_idx, val_idx = train_test_split(
    X_train.index.tolist(),
    test_size=0.2,
    stratify=X_train['mapped'],
    random_state=42
)


photo_path = "/content/Data (1)/Train/"
test_photo_path = "/content/Data (1)/Test/"

train_dataset = ImageDataset(X_train, train_idx, photo_path, transform, train=True)
val_dataset = ImageDataset(X_train, val_idx, photo_path, transform, train=True)
train_full_dataset = ImageDataset(X_train, X_train.index.tolist(), photo_path, transform, train=True)

In [ ]:
test_df = pd.read_csv("test (37).csv", sep=';')
test = test_df.copy()
test_idx = test_df.index.tolist()
test_dataset = ImageDataset(test_df, test_idx, test_photo_path, transform, train=False)

In [ ]:
test_idx = test.index.tolist()

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
train_full_loader = DataLoader(train_full_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [ ]:
def train_model(model, train_loader, val_loader, optimizer, device, epochs=3):
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_f1 = f1_score(train_labels, train_preds, average="macro")
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f}, F1: {train_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_f1 = f1_score(val_labels, val_preds, average="macro")
        print(f"[Epoch {epoch+1}] Val Loss: {val_loss/len(val_loader):.4f}, F1: {val_f1:.4f}")


In [ ]:
def train_model_without_eval(model, train_loader, optimizer, device, epochs=3):
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_f1 = f1_score(train_labels, train_preds, average="macro")
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f}, F1: {train_f1:.4f}")

In [ ]:
def take_from_train_logits(model, train_loader, device):
  final_logits = []
  model.eval()
  with torch.no_grad():
    for images in test_loader:
      images = images.to(DEVICE)
      outputs = model(images)
      final_logits.append(outputs)
  return final_logits

In [ ]:
def eval_model(model, test_loader, device):
  final_logits = []
  model.eval()
  with torch.no_grad():
    for images in test_loader:
      images = images.to(DEVICE)
      outputs = model(images)
      final_logits.append(outputs)

  all_outputs = torch.cat(final_logits, dim=0)
  overall_preds = all_outputs.argmax(dim=1).cpu().numpy()
  return final_logits, overall_preds

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = EfficientNet(num_classes=8).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_model_without_eval(model, train_full_loader, optimizer, device=DEVICE)


In [ ]:
logits_train_by_EN = take_from_train_logits(model, train_full_loader, DEVICE)

In [ ]:
logits_by_EN, preds_by_EN = eval_model(model, test_loader, DEVICE)

In [ ]:
pd.DataFrame(torch.cat(logits_by_EN, dim=0).cpu().numpy()).to_csv('EfficientNet_Medium_eval_logits.csv')



In [ ]:
logits_train_by_RN = take_from_train_logits(model, train_full_loader, DEVICE)

In [ ]:
logits_by_RN, preds_by_RN = eval_model(model, test_loader, DEVICE)

## Вот полный код для копирования

In [ ]:
import torchvision.models as models
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision import transforms
import torch.optim as optim
import os
import pandas as pd
from sklearn.metrics import f1_score

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, indices, photo_path, transform, train=True):
        self.dataframe = dataframe
        self.transform = transform
        self.train = train
        self.indices = indices
        self.photo_path = photo_path

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        original_idx = self.indices[idx]
        filename = self.dataframe.loc[original_idx, 'id']
        file_path = os.path.join(self.photo_path, str(filename))

        image = Image.open(file_path).convert("RGB")
        image = self.transform(image)

        if self.train:
            label = torch.tensor(self.dataframe['mapped'].loc[original_idx], dtype=torch.long)
            return image, label
        else:
            return image
        
transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
        ])
class Resnet(nn.Module):
    def __init__(self, num_classes=8):
        super(Resnet, self).__init__()

        self.model = models.resnet152(weights=models.ResNet152_Weights.IMAGENET1K_V2)

        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

class EfficientNet(nn.Module):
  def __init__(self, num_classes=8):
    super(EfficientNet, self).__init__()

    self.model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.IMAGENET1K_V1)

    self.model.classifier[1] = nn.Linear(self.model.classifier[1].in_features, num_classes)
  def forward(self, x):
    return self.model(x)
class EfficientNetLarge(nn.Module):
    def __init__(self, num_classes=8):
        super(EfficientNetLarge, self).__init__()

        self.model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)

        in_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.model(x)
    
    from sklearn.model_selection import train_test_split
# X_train -- ТВОЙ ДАТАСЕТ
train_idx, val_idx = train_test_split(
    X_train.index.tolist(),
    test_size=0.2,
    stratify=X_train['mapped'],
    random_state=42
)


photo_path = "/content/Data (1)/Train/"
test_photo_path = "/content/Data (1)/Test/"

train_dataset = ImageDataset(X_train, train_idx, photo_path, transform, train=True)
val_dataset = ImageDataset(X_train, val_idx, photo_path, transform, train=True)
train_full_dataset = ImageDataset(X_train, X_train.index.tolist(), photo_path, transform, train=True)
test_df = pd.read_csv("test (37).csv", sep=';')
test = test_df.copy()
test_idx = test_df.index.tolist()
test_dataset = ImageDataset(test_df, test_idx, test_photo_path, transform, train=False)


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
train_full_loader = DataLoader(train_full_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


def train_model(model, train_loader, val_loader, optimizer, device, epochs=3):
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_f1 = f1_score(train_labels, train_preds, average="macro")
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f}, F1: {train_f1:.4f}")

        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        val_f1 = f1_score(val_labels, val_preds, average="macro")
        print(f"[Epoch {epoch+1}] Val Loss: {val_loss/len(val_loader):.4f}, F1: {val_f1:.4f}")
def train_model_without_eval(model, train_loader, optimizer, device, epochs=3):
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            train_labels.extend(labels.cpu().numpy())

        train_f1 = f1_score(train_labels, train_preds, average="macro")
        print(f"[Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f}, F1: {train_f1:.4f}")

def take_from_train_logits(model, train_loader, device):
  final_logits = []
  model.eval()
  with torch.no_grad():
    for images in test_loader:
      images = images.to(DEVICE)
      outputs = model(images)
      final_logits.append(outputs)
  return final_logits
def eval_model(model, test_loader, device=DEVICE):
  final_logits = []
  model.eval()
  with torch.no_grad():
    for images in test_loader:
      images = images.to(DEVICE)
      outputs = model(images)
      final_logits.append(outputs)

  all_outputs = torch.cat(final_logits, dim=0)
  overall_preds = all_outputs.argmax(dim=1).cpu().numpy()
  return final_logits, overall_preds

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model = EfficientNet(num_classes=8).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

train_model_without_eval(model, train_full_loader, optimizer, device=DEVICE)

logits_train_by_EN = take_from_train_logits(model, train_full_loader, DEVICE)
